### Preparar ambiente

In [1]:
import enum
print(enum.__file__)  
# standard library location should be something like 
# /usr/local/lib/python3.6/enum.py

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/enum.py


In [2]:
import sys
!{sys.executable} -m pip uninstall -y enum34

In [3]:
!{sys.executable} -m pip install awswrangler

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [4]:
!{sys.executable} -m pip install PyAthena

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


### Conectar con pyathena

In [5]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [6]:
import pandas as pd
import awswrangler as wr
import numpy as np
import calendar
import boto3
s3 = boto3.resource('s3')

In [7]:
import os
import pandas as pd
import zipfile
import shutil
from io import StringIO
import io

### Declaración funciones para Impo-Expo s3

In [8]:
def write_dataframe_to_csv_on_s3(dataframe, bucket, filename):
    """ Write a dataframe to a CSV on S3 """
    # Create buffer
    csv_buffer = StringIO()
    # Write dataframe to buffer
    dataframe.to_csv(csv_buffer, sep=";" , line_terminator='\n')
    # Create S3 object
    s3_resource = boto3.resource("s3")
    # Write buffer to S3 object
    s3_resource.Object(bucket, filename).put(Body=csv_buffer.getvalue())
    print("Writing {} records to {}".format(len(dataframe), filename))
    print("S3")
    print(dataframe.shape)
    

In [9]:
def write_dataframe_to_excel_on_s3(dataframe, bucket, filename):
    """ Write a dataframe to a excel on S3 """
    with io.BytesIO() as output:
        with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
            dataframe.to_excel(writer,index=False)
        data = output.getvalue()

    s3 = boto3.resource('s3')
    s3.Bucket(bucket).put_object(Key=filename, Body=data)
    

In [10]:
def import_dataframe_csv_from_s3(bucket, filename):
    client = boto3.client('s3')
    bucket_name = bucket
    object_key = filename
    csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_string),sep=";")
    return df

In [11]:
def import_dataframe_excel_from_s3(bucket, filename):
    client = boto3.client('s3')
    bucket_name = bucket
    object_key = filename
    obj = client.get_object(Bucket=bucket_name, Key=object_key)
    data = obj['Body'].read()
    df = pd.read_excel(io.BytesIO(data), encoding='utf-8')
    return df

In [12]:
def write_file_to_s3(bucket,file_local,file_s3):
    s3.Bucket(bucket).upload_file(file_local,file_s3)

### Crea y guarda tabla final

In [13]:
pais = 'LATAM'

In [14]:
### Declarando periodo
fecha_inicio = '02-01-2020'
fecha_final = '04-30-2020'
fechas = pd.date_range(fecha_inicio, fecha_final)

In [15]:
### Declarando segundo periodo - Caso discontinuo o actualizacion
fecha_inicio = '09-01-2020'
fecha_final = '09-30-2020'
fechas2 = pd.date_range(fecha_inicio, fecha_final)

#### Datos con filtro tenue - 10p

##### Tabla cities - Periodo de analisis

In [16]:
### Generando tabla
df_final = []
start_time = time.time()
for i,fecha in enumerate(fechas):
    time_fecha = time.time()
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    #print(codigo_fecha)
    tabla_usuarios_fecha = f'historico_cities_{pais}_{codigo_fecha}'
    query = f'''SELECT * FROM {tabla_usuarios_fecha}'''
    #print(f"{codigo_fecha}")
    df = cursor.execute(query).as_pandas()
    if (codigo_fecha=='201'):
        df_final=df
    else:
        df_final=df_final.append(df)
df_final=df_final.rename(columns={'mas_de1km':'MasDe1km','_mas_de1km':'%MasDe1km','mediana1_km':'Mediana1KM','mes':'Mes','dia':'Dia'})
df_final_4 = df_final
tiempo = (time.time()-time_fecha)/60
print((time.time()-start_time)/60)


###revisar columnas de informacion
df_final_4.head()

2.6938851435979205


,Mes,Dia,id_city,name_city,country_city,MasDe1km,n_users,%MasDe1km,Mediana1KM,key4
0,2,1,19,Rosarito,Mexico,803,1119,71.760500,27.928633,201
1,2,1,20,Ensenada,Mexico,3232,5495,58.817106,22.095301,201
2,2,1,23,Las Delicias,Mexico,578,1481,39.027684,19.251624,201
3,2,1,31,Tecate,Mexico,662,1242,53.301127,13.310151,201
4,2,1,43,Cabo San Lucas,Mexico,1540,2630,58.555133,18.524697,201


In [17]:
df_final_4.shape

(89822, 10)

##### Tabla cities - Segundo periodo de analisis - Activar si el periodo es discontinuo

In [18]:
### Generando tabla
df_final = []
start_time = time.time()
for i,fecha in enumerate(fechas2):
    time_fecha = time.time()
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    #print(codigo_fecha)
    tabla_usuarios_fecha = f'historico_cities_{pais}_{codigo_fecha}'
    query = f'''SELECT * FROM {tabla_usuarios_fecha}'''
    #print(f"{codigo_fecha}")
    df = cursor.execute(query).as_pandas()
    if (codigo_fecha=='901'):
        df_final=df
    else:
        df_final=df_final.append(df)
df_final=df_final.rename(columns={'mas_de1km':'MasDe1km','_mas_de1km':'%MasDe1km','mediana1_km':'Mediana1KM','mes':'Mes','dia':'Dia'})
df_empalme_4 = df_final
tiempo = (time.time()-time_fecha)/60
print((time.time()-start_time)/60)

###juntando base de datos
df_final_4 = df_final_4.append(df_empalme_4, ignore_index=True)

###revisar columnas de informacion
df_final_4.head()

0.8562481681505839


,Mes,Dia,id_city,name_city,country_city,MasDe1km,n_users,%MasDe1km,Mediana1KM,key4
0,2,1,19,Rosarito,Mexico,803,1119,71.760500,27.928633,201
1,2,1,20,Ensenada,Mexico,3232,5495,58.817106,22.095301,201
2,2,1,23,Las Delicias,Mexico,578,1481,39.027684,19.251624,201
3,2,1,31,Tecate,Mexico,662,1242,53.301127,13.310151,201
4,2,1,43,Cabo San Lucas,Mexico,1540,2630,58.555133,18.524697,201


In [19]:
df_final_4.shape

(119739, 10)

##### Ajustes finales

In [20]:
df_final_4=df_final_4.sort_values(by=['country_city','id_city','Mes','Dia'])
df_final_4.head()

,Mes,Dia,id_city,name_city,country_city,MasDe1km,n_users,%MasDe1km,Mediana1KM,key4
560,2,1,962,San Juan,Argentina,1164,1659,70.162749,13.953333,201
1557,2,2,962,San Juan,Argentina,1117,1658,67.370326,11.632643,202
2557,2,3,962,San Juan,Argentina,1204,1756,68.564920,13.081143,203
3555,2,4,962,San Juan,Argentina,1257,1792,70.145089,13.599713,204
4553,2,5,962,San Juan,Argentina,1299,1782,72.895623,13.287493,205


In [21]:
df_final_4.shape

(119739, 10)

##### Guardar archivos

In [22]:
nivel_4 = 'cities'
ruta=f'graphdata-mobility-public/Results_cities/{pais}/almenos10p/{pais}_{nivel_4}_10p.csv'
write_dataframe_to_csv_on_s3(df_final_4, 'iadbprod-csd-hub-analyticaldata', ruta)

Writing 119739 records to graphdata-mobility-public/Results_cities/LATAM/almenos10p/LATAM_cities_10p.csv
S3
(119739, 10)


#### Datos con filtro severo - Base Usuarios

##### Tabla cities - Periodo de analisis

In [23]:
### Generando tabla
df_final = []
start_time = time.time()
for i,fecha in enumerate(fechas):
    time_fecha = time.time()
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    #print(codigo_fecha)
    tabla_usuarios_fecha = f'historico_cities_{pais}_{codigo_fecha}_fs'
    query = f'''SELECT * FROM {tabla_usuarios_fecha}'''
    #print(f"{codigo_fecha}")
    df2 = cursor.execute(query).as_pandas()
    if (codigo_fecha=='201'):
        df_final=df2
    else:
        df_final=df_final.append(df2)
df_final=df_final.rename(columns={'mas_de1km':'MasDe1km','_mas_de1km':'%MasDe1km','mediana1_km':'Mediana1KM','mes':'Mes','dia':'Dia'})
df_final_fs4 = df_final
tiempo = (time.time()-time_fecha)/60
print((time.time()-start_time)/60)

###revisar columnas de informacion
df_final_fs4.head()

2.761206372578939


,Mes,Dia,id_city,name_city,country_city,MasDe1km,n_users,%MasDe1km,Mediana1KM,key4
0,2,1,19,Rosarito,Mexico,394,496,79.435484,25.973188,201
1,2,1,20,Ensenada,Mexico,1571,2057,76.373359,20.406825,201
2,2,1,23,Las Delicias,Mexico,160,283,56.537102,22.607504,201
3,2,1,31,Tecate,Mexico,321,429,74.825175,11.616490,201
4,2,1,43,Cabo San Lucas,Mexico,677,939,72.097977,19.786782,201


In [24]:
df_final_fs4.shape

(89424, 10)

##### Tabla cities - Segundo periodo de analisis - Activar si el periodo es discontinuo

In [25]:
### Generando tabla
df_final = []
start_time = time.time()
for i,fecha in enumerate(fechas2):
    time_fecha = time.time()
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    #print(codigo_fecha)
    tabla_usuarios_fecha = f'historico_cities_{pais}_{codigo_fecha}_fs'
    query = f'''SELECT * FROM {tabla_usuarios_fecha}'''
    #print(f"{codigo_fecha}")
    df2 = cursor.execute(query).as_pandas()
    if (codigo_fecha=='901'):
        df_final=df2
    else:
        df_final=df_final.append(df2)
df_final=df_final.rename(columns={'mas_de1km':'MasDe1km','_mas_de1km':'%MasDe1km','mediana1_km':'Mediana1KM','mes':'Mes','dia':'Dia'})
df_empalme_fs4 = df_final
tiempo = (time.time()-time_fecha)/60
print((time.time()-start_time)/60)

###juntando base de datos
df_final_fs4 = df_final_fs4.append(df_empalme_fs4, ignore_index=True)

###revisar columnas de informacion
df_final_fs4.head()

0.9377510627110799


,Mes,Dia,id_city,name_city,country_city,MasDe1km,n_users,%MasDe1km,Mediana1KM,key4
0,2,1,19,Rosarito,Mexico,394,496,79.435484,25.973188,201
1,2,1,20,Ensenada,Mexico,1571,2057,76.373359,20.406825,201
2,2,1,23,Las Delicias,Mexico,160,283,56.537102,22.607504,201
3,2,1,31,Tecate,Mexico,321,429,74.825175,11.616490,201
4,2,1,43,Cabo San Lucas,Mexico,677,939,72.097977,19.786782,201


In [26]:
df_final_fs4.shape

(118882, 10)

##### Ajustes finales

In [27]:
df_final_fs4=df_final_fs4.sort_values(by=['country_city','id_city','Mes','Dia'])
df_final_fs4.head()

,Mes,Dia,id_city,name_city,country_city,MasDe1km,n_users,%MasDe1km,Mediana1KM,key4
554,2,1,962,San Juan,Argentina,648,825,78.545455,16.149753,201
1547,2,2,962,San Juan,Argentina,606,809,74.907293,14.623176,202
2540,2,3,962,San Juan,Argentina,660,871,75.774971,15.355668,203
3535,2,4,962,San Juan,Argentina,694,881,78.774120,16.953672,204
4529,2,5,962,San Juan,Argentina,718,908,79.074890,15.859436,205


In [28]:
df_final_fs4.shape

(118882, 10)

##### Guardar archivos

In [29]:
nivel_4 = 'cities'
ruta=f'graphdata-mobility-public/Results_cities/{pais}/base_usuario/{pais}_{nivel_4}_baseuser.csv'
write_dataframe_to_csv_on_s3(df_final_fs4, 'iadbprod-csd-hub-analyticaldata', ruta)

Writing 118882 records to graphdata-mobility-public/Results_cities/LATAM/base_usuario/LATAM_cities_baseuser.csv
S3
(118882, 10)
